In [8]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

In [9]:
#loading datasets
df_train = pd.read_csv('train.csv')
df_test  = pd.read_csv('test.csv')
#concatenation
df_combined = pd.concat([df_train, df_test])

C:\Users\Abhishek\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [10]:
#dividing train and test data
df_drop_target=df_combined.drop(['Approved'], axis=1)
Xtrain = df_drop_target.head(69713)
Xtest = df_drop_target.tail(30037)
Xtarget=df_train['Approved']

In [11]:
X_train, X_test, y_train, y_test= train_test_split(Xtrain, Xtarget, test_size=0.3, random_state=42)

# Scalar

In [ ]:
X is independent features/ predictors
y is target variable
# scaling of target variable is not required 

ss = StandardScaler().fit(X,y)
X_ = ss.transform(X)

# Logistic Regression

In [17]:
#this is not working bc data is not cleaned
'''logreg = LogisticRegression()
logreg.fit(X_train, y_train)'''

'logreg = LogisticRegression()\nlogreg.fit(X_train, y_train)'

In [ ]:
#### base model

model_cl = LogisticRegression()
model_cl = model_cl.fit(X_train, y_train) #### training 
predict_train = model_cl.predict(X_test)
print(accuracy_score(y_train, model_cl.predict(X_train))) # accuracy on train
accuracy_score(y_test, model_cl.predict(X_test)) # accuracy on cross validation

### prediction on the test.csv
y_test2 = model_cl.predict(X_test_)

# test2 is the set dataset.. not having target variable
test2['Survived'] = y_test2

### save dataframe as csv file
# test2[['PassengerId','Survived']].to_csv('submission.csv', index = False)

model_cl

### Logistic Regression Model Tuning

In [ ]:
# model_cl = LogisticRegression()

param = {'C':[0.1,1,10],
        'penalty':['l1','l2'] }#PENALTIES ARE USED TO SHRINK THE CORRELATED FEATURES WITHOUT REMOVING IN L2
                               # L1 PENALTY IS USED TO SHRINK THE USELESS FEATURE TO ZERO, 
                               # USED IN FEATURE ENGINEERING AND IMPORTANT FEATURE SELECTION
# Using GRID SEARCH CV
gs = GridSearchCV(estimator=model_cl, param_grid=param, scoring='accuracy', cv = 3,
                 n_jobs= -1)

#  Grid Search 
## fiting
gs = gs.fit(X_train,y_train)
gs.best_score_

gs.best_params_

## accuracy on validation dataset
accuracy_score(y_test, gs.predict(X_test))

pd.DataFrame(gs.cv_results_).sort_values(by = ['mean_test_score'], ascending=False)

# Decision Tree

In [ ]:
'''dtree = DecisionTreeClassifier(criterion = "gini", random_state = 100,max_depth=5, min_samples_leaf=5)
dtree.fit(X_train, y_train)'''

# Train Accuracy / Score

In [ ]:
#for logistic regression 
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [ ]:
#for decision tree
y_pred_tree=dtree.predict(X_test)
print('Accuracy of Decision tree classifier on test set: {:.2f}'.format(dtree.score(X_test, y_test)))

In [ ]:
# Importing confusion matrix for getting the exact number of times positive and and negatives are wrongly classified
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

In [ ]:
#Importing classification Report for finding what is the accuracy,precision,recall of the model based on classification tree
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_tree))

# Importing ruc_auc_score and roc_curve from sklearn

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
tree_roc_auc=roc_auc_score(y_test, dtree.predict(X_test))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
fpr, tpr, thresholds = roc_curve(y_test, dtree.predict_proba(X_test)[:,1])

plt.figure()

plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot(fpr, tpr, label='Tree Regression (area = %0.2f)' % tree_roc_auc)

plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

In [ ]:
'''As seen from the auc of both logistic regression and Decision tree 
curves are overlapping hence they are same'''

### PREDICTIONS FOR UNSEEN DATA

In [ ]:
y_prediction = logreg.predict(Xtest)

In [ ]:
y_prediction_tree=dtree.predict(Xp)

## IMPORTING PREDICTED COLUMN INTO THE TEST DATASET

In [ ]:
df_test['Approved']=y_prediction
df_test.head()

# RF, pipeline, condusion matrix

In [ ]:
steps = [('over', RandomOverSampler()), ('model', RandomForestClassifier())] ### steps
#steps = [('under', RandomUnderSampler()), ('model', RandomForestClassifier())]
#steps = [('over', SMOTE(random_state= 42)), ('model', RandomForestClassifier())]


pipeline = Pipeline(steps=steps) ### object

pipeline.fit(X_train, y_train) ## training

y_hat = pipeline.predict(X_test) ## prediction
print('Validation set: confusion matrix with random-over-sampling')
cm = confusion_matrix(y_test, y_hat, labels=[0,1])
annot_kws = {"ha": 'left',"va": 'top'}
print(cm)
sns.heatmap(cm, annot=True, center = True,fmt="d",linewidths=0.5,cmap="Greens",vmin=0,vmax = cm.max(),annot_kws=annot_kws)

In [ ]:
# Using for loop
for k in range(4,10,1):
    steps = [('over', SMOTE(random_state=42, k_neighbors=k)), ('model', RandomForestClassifier())]
    pipeline = Pipeline(steps=steps)
    pipeline.fit(X_train, y_train) ## training
    y_hat = pipeline.predict(X_test) ## prediction
    print('validation set:AUC Score: {}%'.format(round(roc_auc_score(y_test, y_hat)*100,4)))
    print('Validation set: confusion matrix with SMOTE-over-sampling with k = {}'.format(k))
    print(confusion_matrix(y_test, y_hat, labels=[0,1]))

# AUC score

In [ ]:
print('Training AUC Score: {}'.format(round(roc_auc_score(y_train, pipeline.predict(X_train))*100,4)))
print('validation AUC Score: {}'.format(round(roc_auc_score(y_test, y_hat)*100,4)))

In [ ]:
###  searching best k value 
 - combining over-sampling and under-sampling
 - tunning k value in K_neighbours
# values to evaluate
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
k_values = [1, 2, 3, 4, 5, 6, 7]
for k in k_values:
# define pipeline
    model = RandomForestClassifier()
    over  = SMOTE(sampling_strategy=0.1, k_neighbors=k)
    under = RandomUnderSampler(sampling_strategy=0.5)
    steps = [('over', over), ('under', under), ('model', model)]
    pipeline = Pipeline(steps=steps)
    # evaluate pipeline
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(pipeline, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
    score = np.mean(scores)
    print('> k=%d, Mean ROC AUC: %.3f' % (k, score))

In [ ]:
steps = [('over', RandomOverSampler()), ('model', LogisticRegression())] ### steps

pipeline = Pipeline(steps=steps) ### object

pipeline.fit(X_train, y_train) ## training

y_hat = pipeline.predict(X_test) ## prediction
print('Validation set: confusion matrix with random-over-sampling')
cm = confusion_matrix(y_test, y_hat, labels=[0,1])
annot_kws = {"ha": 'left',"va": 'top'}
sns.heatmap(cm, annot=True, center = True,fmt="d",linewidths=0.5,cmap="Greens",vmin=0,vmax = cm.max(),annot_kws=annot_kws)

print('Training AUC Score: {}'.format(round(roc_auc_score(y_train, pipeline.predict(X_train))*100,4)))
print('validation AUC Score: {}'.format(round(roc_auc_score(y_test, y_hat)*100,4)))

In [ ]:
# values to evaluate
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
k_values = [1, 2, 3, 4, 5, 6, 7]
for k in k_values:
	# define pipeline
	model = RandomForestClassifier()
	over  = SMOTE(sampling_strategy=0.1, k_neighbors=k)
	under = RandomUnderSampler(sampling_strategy=0.5)
	steps = [('over', over), ('under', under), ('model', model)]
	pipeline = Pipeline(steps=steps)
	# evaluate pipeline
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(pipeline, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
	score = np.mean(scores)
	print('> k=%d, Mean ROC AUC: %.3f' % (k, score))

# Random Forest Classifier usin GridSearch

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
#### Randomforest classifier
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)
print(accuracy_score(y_train, model_rf.predict(X_train)))
print(accuracy_score(y_test, model_rf.predict(X_test)))



In [ ]:
#### Randomforestclassifier using Gridsearch cv

param = {'max_depth':[1,2,3,4],
        'max_features':['auto','sqrt'] }

In [ ]:
gs_r = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param, scoring='accuracy', cv = 3,
                 n_jobs= -1)
#estimator cab be LR, decision tree etc

In [4]:
gs_r = gs_r.fit(X_train, y_train)

gs_r.best_score_

gs_r.best_params_

print(accuracy_score(y_train, gs_r.predict(X_train)))
print(accuracy_score(y_test, gs_r.predict(X_test)))


NameError: name 'gs_r' is not defined

### Random Forest classifier Model Tuning

In [ ]:
param = {'max_depth':[1,2,3,4],
        'max_features':['auto','sqrt'] }

gs_r = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param, scoring='accuracy', cv = 10,
                 n_jobs= 2)

gs_r = gs_r.fit(X_train, y_train)
print(accuracy_score(y_train, gs_r.predict(X_train)))
print(accuracy_score(y_test, gs_r.predict(X_test)))

#prediction on the test.csv
y_testsetRF = gs_r.predict(X_test_)
print(X_test_.shape)
print(y_testsetRF.shape)
test2.shape

test2['Survived'] = y_testsetRF
test2.head()

### save dataframe as csv file
test2[['PassengerId','Survived']].to_csv('submission.csv', index = False)

pd.DataFrame(gs_r.cv_results_).sort_values(by = ['mean_test_score'], ascending=False)

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

# Decision Tree

In [ ]:
# Decision tree with depth = 2
model_dt_2 = DecisionTreeClassifier(random_state=1, max_depth=2)
model_dt_2.fit(X_train, y_train)
model_dt_2_score_train = model_dt_2.score(X_train, y_train)
print("Training score: ",model_dt_2_score_train)
model_dt_2_score_test = model_dt_2.score(X_test, y_test)
print("Testing score: ",model_dt_2_score_test)
#y_pred_dt = model_dt_2.predict_proba(x_test)[:, 1]



#Decision tree

model_dt = DecisionTreeClassifier(max_depth = 8, criterion ="entropy")
model_dt.fit(X_train, y_train)
y_pred_dt = model_dt.predict_proba(X_test)[:, 1]



predictions = model_dt.predict(X_test)

# Model Accuracy
print (model_dt.score(X_test, y_test))
y_actual_result = y_test


print (confusion_matrix(y_test, predictions))



accuracy_score(y_test, predictions)

# area under curve

In [ ]:
fpr_dt, tpr_dt, _ = roc_curve(y_test, y_pred_dt)
roc_auc_dt = auc(fpr_dt, tpr_dt)



plt.figure(1)
lw = 2
plt.plot(fpr_dt, tpr_dt, color='green',
         lw=lw, label='Decision Tree(AUC = %0.2f)' % roc_auc_dt)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Area Under Curve')
plt.legend(loc="lower right")
plt.show()

# SVM 

In [ ]:
from sklearn.svm import SVC
from sklearn import metrics
svc=SVC() #Default hyperparameters
svc.fit(X_train,y_train)
y_pred=svc.predict(X_test)
print('Accuracy Score:')
print(metrics.accuracy_score(y_test,y_pred))



# linear kernal
svc=SVC(kernel='linear')
'''# polynomial kernal
svc=SVC(kernel='poly')

# radial kernal
svc=SVC(kernel='rbf')
'''
svc.fit(X_train,y_train)
y_pred=svc.predict(X_test)
print('Accuracy Score:')
print(metrics.accuracy_score(y_test,y_pred))



